## 3.6　层级索引

我们经常会遇到存储多维数据的需求，数据索引超过一两个键。因此，Pandas 提供了Panel 和Panel4D 对象解决三维数据与四维数据（详情请参见3.7 节）。
<br>而在实践中，更直观的形式是通过**层级索引（hierarchical indexing，也被称为多级索引，multi-indexing）**配合多个有不同等级（level）的一级索引一起使用，这样就可以将高维数组转换成类似一维Series 和二维DataFrame 对象的形式。

在这一节中，我们将介绍创建MultiIndex 对象的方法，多级索引数据的取值、切片和统计值的计算，以及普通索引与层级索引的转换方法。

In [1]:
import pandas as pd
import numpy as np

### 3.6.1　多级索引Series

用一维的Series 对象表示二维数据——用一系列包含特征与数值的数据点来简单演示。

**1. 笨办法**

假设你想要分析美国各州在两个不同年份的数据。如果你用前面介绍的Pandas 工具来处理，那么可能会用一个Python 元组来表示索引：

In [3]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

通过元组构成的多级索引，你可以直接在Series 上取值或用切片查询数据：

In [4]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

但是这么做很不方便。假如你想要选择所有2000 年的数据，那么就得用一些比较复杂的（可能也比较慢的）清理方法了：

In [5]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

**2. 好办法：Pandas多级索引**

用元组表示索引其实是多级索引的基础，Pandas的MultiIndex 类型提供了更丰富的操作方法。我们可以用元组创建一个多级索引，如下所示：

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

现MultiIndex 里面有一个levels 属性表示索引的等级——这样做可以将州名和年份作为每个数据点的不同标签。

如果将前面创建的pop 的索引重置（reindex）为MultiIndex，就会看到层级索引：

In [8]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

其中前两列表示Series 的多级索引值，第三列是数据。你会发现有些行仿佛缺失了第一列数据——这其实是多级索引的表现形式，每个空格与上面的索引相同。

现在可以直接用第二个索引获取2010 年的全部数据，与Pandas 的切片查询用法一致：

In [9]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

结果是单索引的数组，正是我们需要的。
<br>与之前的元组索引相比，多级索引的语法更简洁。（操作也更方便！）

**3. 高维数据的多级索引**

我们其实完全可以用一个带行列索引的简单DataFrame 代替前面的多级索引。
<br>其实Pandas 已经实现了类似的功能。`unstack()` 方法可以快速将一个多级索引的Series 转化为普通索引的DataFrame：

In [10]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


当然了，也有stack() 方法实现相反的效果：

In [11]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

你可能会纠结于为什么要费时间研究层级索引。其实理由很简单：
> 如果我们可以用含多级索引的一维Series 数据表示二维数据，那么我们就可以用Series 或DataFrame 表示三维甚至更高维度的数据。多级索引每增加一级，就表示数据增加一维，利用这一特点就可以轻松表示任意维度的数据了。

假如要增加一列显示每一年各州的人口统计指标（例如18岁以下的人口），那么对于这种带有MultiIndex 的对象，增加一列就像DataFrame 的操作一样简单：

In [12]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

另外，所有在3.4 节介绍过的通用函数和其他功能也同样适用于层级索引。
<br>我们可以计算上面数据中18 岁以下的人口占总人口的比例：

In [13]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


In [14]:
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

### 3.6.2　多级索引的创建方法

为Series 或DataFrame 创建多级索引最直接的办法就是将index 参数设置为至少二维的索引数组,MultiIndex 的创建工作将在后台完成。如下所示：

In [15]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.670483  0.122595
  2  0.417662  0.775548
b 1  0.860182  0.151973
  2  0.919772  0.229444

同理，如果你把将元组作为键的字典传递给Pandas， Pandas 也会默认转换为MultiIndex：

In [16]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

但是有时候显式地创建MultiIndex 也是很有用的，下面来介绍一些创建方法。

#### 3.6.2.1. 显式地创建多级索引

可以用pd.MultiIndex 中的类方法更加灵活地构建多级索引。
<br>可以通过一个有不同等级的若干简单数组组成的列表来构建MultiIndex：

In [26]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

也可以通过包含多个索引值的元组构成的列表创建MultiIndex：

In [27]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

还可以用两个索引的笛卡尔积（Cartesian product）创建MultiIndex：

In [28]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

更可以直接提供levels（包含每个等级的索引值列表的列表）和labels（包含每个索引值标签列表的列表）创建MultiIndex：

In [29]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

在创建Series 或DataFrame 时，可以将这些对象作为index 参数，或者通过reindex 方法更新Series 或DataFrame 的索引。

#### 3.6.2.2 多级索引的等级名称

给MultiIndex 的等级加上名称会为一些操作提供便利。
<br>你可以在前面任何一个MultiIndex构造器中通过names 参数设置等级名称，也可以在创建之后通过索引的names 属性来修改名称：

In [30]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

在处理复杂的数据时，为等级设置名称是管理多个索引值的好办法。

#### 3.6.2.3 多级列索引

每个DataFrame 的行与列都是对称的，也就是说既然有多级行索引，那么同样可以有多级列索引。
<br>让我们通过一份医学报告的模拟数据来演示：

In [31]:
# 多级行列索引
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])
# 模拟数据
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# 创建DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  38.7  34.0  38.0  28.0  38.0
     2      53.0  38.4  62.0  36.1  28.0  36.4
2014 1      34.0  36.0  48.0  38.4  31.0  38.6
     2      31.0  37.4  21.0  37.2  26.0  35.8

多级行列索引的创建非常简单。上面创建了一个简易的四维数据，四个维度分别为被检查人的姓名、检查项目、检查年份和检查次数。可以在列索引的第一级查询姓名，从而获取包含一个人（例如Guido）全部检查信息的DataFrame：

In [32]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      34.0  38.0
     2      62.0  36.1
2014 1      48.0  38.4
     2      21.0  37.2

如果想获取包含多种标签的数据，需要通过对多个维度（姓名、国家、城市等标签）的多次查询才能实现，这时使用多级行列索引进行查询会非常方便。

### 3.6.3　多级索引的取值与切片

对MultiIndex 的取值和切片操作很直观，你可以直接把索引看成额外增加的维度。
<br>我们先来介绍Series 多级索引的取值与切片方法，再介绍DataFrame 的用法。

#### 3.6.3.1. Series多级索引

看看下面由各州历年人口数量创建的多级索引Series：

In [33]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

可以通过对多个级别索引值获取单个元素：

In [34]:
pop['California', 2000]

33871648

MultiIndex 也支持**局部取值（partial indexing）**，即只取索引的某一个层级。假如只取最高级的索引，获得的结果是一个新的Series，未被选中的低层索引值会被保留：

In [36]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

类似的还有**局部切片**，不过要求MultiIndex 是按顺序排列的（就像将在3.6.4 节介绍的那样）：

In [37]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

如果索引已经排序，那么可以用较低层级的索引取值，第一层级的索引可以用空切片：

In [38]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

其他取值与数据选择的方法（详情请参见3.3 节）也都起作用。
<br>下面的例子是通过布尔掩码选择数据：

In [39]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

也可以用花哨的索引选择数据：

In [40]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 3.6.3.2 DataFrame多级索引

 DataFrame 多级索引的用法与Series 类似。

In [41]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  38.7  34.0  38.0  28.0  38.0
     2      53.0  38.4  62.0  36.1  28.0  36.4
2014 1      34.0  36.0  48.0  38.4  31.0  38.6
     2      31.0  37.4  21.0  37.2  26.0  35.8

由于DataFrame 的基本索引是列索引，因此Series 中多级索引的用法到了DataFrame 中就应用在列上了。
<br>例如，可以通过简单的操作获取Guido 的心率数据：

In [42]:
health_data['Guido', 'HR']

year  visit
2013  1        34.0
      2        62.0
2014  1        48.0
      2        21.0
Name: (Guido, HR), dtype: float64

与单索引类似，在3.3 节介绍的loc、iloc 和ix 索引器都可以使用，例如：

In [43]:
health_data.iloc[:2, :2]
# 应该是针对数据来看的

subject      Bob      
type          HR  Temp
year visit            
2013 1      37.0  38.7
     2      53.0  38.4

虽然这些索引器将多维数据当作二维数据处理，但是在loc 和iloc 中可以传递多个层级的索引元组，例如：

In [44]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        37.0
      2        53.0
2014  1        34.0
      2        31.0
Name: (Bob, HR), dtype: float64

这种索引元组的用法不是很方便，如果在元组中使用切片还会导致语法错误：

In [45]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-45-fb34fa30ac09>, line 1)

虽然你可以用Python 内置的`slice()` 函数获取想要的切片，但是还有一种更好的办法，就是使用IndexSlice对象。Pandas 专门用它解决这类问题，例如：

In [46]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,37.0,34.0,28.0
2014,1,34.0,48.0,31.0


### 3.6.4　多级索引行列转换

使用多级索引的**关键**是**掌握有效数据转换的方法**。Pandas 提供了许多操作，可以让数据在内容保持不变的同时，按照需要进行行列转换。
<br>之前我们用一个简短的例子演示过`stack()` 和`unstack()` 的用法，但其实还有许多合理控制层级行列索引的方法。

#### 3.6.4.1 有序的索引和无序的索引
在前面的内容里，我们曾经简单提过多级索引排序，这里需要详细介绍一下。
<br>如果MultiIndex不是有序的索引，那么大多数切片操作都会失败。

首先创建一个不按字典顺序（lexographically）排列的多级索引Series：

In [47]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.930695
      2      0.406086
c     1      0.345412
      2      0.264919
b     1      0.744301
      2      0.276310
dtype: float64

如果想对索引使用局部切片，那么错误就会出现：

In [48]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


尽管从错误信息里面看不出具体的细节，但问题是出在**MultiIndex 无序排列**上。
<br>局部切片和许多其他相似的操作都要求MultiIndex 的各级索引是**有序**的（即按照字典顺序由A 至Z）。
<br>为此，Pandas 提供了许多便捷的操作完成排序，如`sort_index()` 和`sortlevel()` 方法。
<br>我们用最简单的`sort_index()` 方法来演示：

In [49]:
data = data.sort_index()
data

char  int
a     1      0.930695
      2      0.406086
b     1      0.744301
      2      0.276310
c     1      0.345412
      2      0.264919
dtype: float64

索引排序之后，局部切片就可以正常使用了：

In [50]:
data['a':'b']

char  int
a     1      0.930695
      2      0.406086
b     1      0.744301
      2      0.276310
dtype: float64

#### 3.6.4.2 索引stack与unstack
前文曾提过，我们可以将一个多级索引数据集转换成简单的二维形式，可以通过level 参数设置转换的索引层级：

In [53]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [52]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [54]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


unstack() 是stack() 的逆操作，同时使用这两种方法让数据保持不变：

In [55]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

#### 3.6.4..3 索引的设置与重置

层级数据维度转换的另一种方法是**行列标签转换**，可以通过`reset_index` 方法实现。
<br>如果在上面的人口数据Series 中使用该方法，则会生成一个列标签中包含之前行索引标签state 和year 的DataFrame。也可以用数据的name 属性为列设置名称：

In [56]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


在解决实际问题的时候，如果能将类似这样的原始输入数据的列直接转换成MultiIndex，
通常将大有裨益。其实可以通过DataFrame 的set_index 方法实现，返回结果就会是一个
带多级索引的DataFrame：

In [57]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

### 3.6.5　多级索引的数据累计方法

前面我们已经介绍过一些Pandas 自带的数据累计方法，比如`mean()`、`sum()` 和`max()`。而对于层级索引数据，可以设置参数level 实现对数据子集的累计操作。

再一次以体检数据为例：

In [58]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      37.0  38.7  34.0  38.0  28.0  38.0
     2      53.0  38.4  62.0  36.1  28.0  36.4
2014 1      34.0  36.0  48.0  38.4  31.0  38.6
     2      31.0  37.4  21.0  37.2  26.0  35.8

如果你需要计算每一年各项指标的平均值，那么可以将参数level 设置为索引year：

In [59]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido          Sue      
type       HR   Temp    HR   Temp    HR  Temp
year                                         
2013     45.0  38.55  48.0  37.05  28.0  37.2
2014     32.5  36.70  34.5  37.80  28.5  37.2

如果再设置axis 参数，就可以对列索引进行类似的累计操作了：

In [60]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,40.333333,37.600000
2014,31.833333,37.233333


通过这两行数据，我们就可以获取每一年所有人的平均心率和体温了。
<br>这种语法其实就是GroupBy 功能的快捷方式，我们将在3.9 节详细介绍。

### Panel 数据

这里还有一些Pandas 的基本数据结构没有介绍到，包括pd.Panel 对象和pd.Panel4D对象。这两种数据结构可以分别看成是（一维数组）Series 和（二维数组）DataFrame的三维与四维形式。如果你熟悉Series 和DataFrame 的使用方法，那么Panel 和Panel4D 使用起来也会很简单，ix、loc 和iloc 索引器（详情请参见3.3 节）在高维数据结构上的用法更是完全相同。

但是本书并不打算进一步介绍这两种数据结构，我个人认为多级索引在大多数情况下都是更实用、更直观的高维数据形式。另外，Panel 采用密集数据存储形式，而多级索引采用稀疏数据存储形式。在解决许多真实的数据集时，随着维度的不断增加，密集数据存储形式的效率将越来越低。但是这类数据结构对一些有特殊需求的应用还是有用的。如果你想对Panel 与Panel4D 数据结构有更多的认识，请参见3.14 节。